In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

In [ ]:
# download bdd100k to drive first
!unzip bdd100k_images.zip

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class BDDDataset(Dataset):
  def __init__(self, fnames, captions, transform=None):
    self.fnames = fnames
    self.captions = captions
    self.transform = transform

  def __len__(self):
    return len(self.fnames)
  
  def __getitem__(self, i):
    img, captions = Image.open(self.fnames[i]), self.captions[self.fnames[i]]
    if self.transform is not None:
      img = self.transform(img)
    return img, captions 

def collate(batch):
  imgs, captions = torch.stack([x[0] for x in batch]), [x[1] for x in batch]
  return imgs, captions

In [ ]:
import clip
import torch
from tqdm.notebook import tqdm

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

In [ ]:
import glob 
train_10k = [f for f in glob.glob("./bdd100k/images/10k/train/*.jpg")]
test_10k = [f for f in glob.glob("./bdd100k/images/10k/test/*.jpg")]
val_10k = [f for f in glob.glob("./bdd100k/images/10k/val/*.jpg")]
train_100k = [f for f in glob.glob("./bdd100k/images/100k/train/*.jpg")]
test_100k = [f for f in glob.glob("./bdd100k/images/100k/test/*.jpg")]
val_100k = [f for f in glob.glob("./bdd100k/images/100k/val/*.jpg")]
all_filenames = train_10k + test_10k + val_10k + train_100k + test_100k + val_100k

In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class BDDDataset(Dataset):
  def __init__(self, fnames, transform=None):
    self.fnames = fnames
    self.transform = transform

  def __len__(self):
    return len(self.fnames)
  
  def __getitem__(self, i):
    img = Image.open(self.fnames[i])
    if self.transform is not None:
      img = self.transform(img)
    return img

In [ ]:
dataset = BDDDataset(all_filenames, preprocess)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, num_workers=8, shuffle=False)

In [ ]:
fin_features = []
with torch.no_grad():
  for idx, images in tqdm(enumerate(dataloader)):
    images = images.to(device)
    image_features = model.encode_image(images)
    fin_features.append(image_features)
fin_features = torch.cat(fin_features)
torch.save(fin_features, './bdd_fin_features.pt')